In [2]:
%load_ext autoreload
%autoreload 2

import pathlib
from datetime import timedelta
from typing import List, Dict
import random

import pandas as pd
pd.set_option('display.max_rows', 200)
import numpy as np

from tqdm import tqdm

import plotly.express as px
import plotly.graph_objects as go
import plotly.subplots as sp

from data_handlers.runtimes import *

In [7]:
experiment_path = pathlib.Path(f"/home/sterz/maci-docker-compose/maci_data/binary_files")
experiment_ids = [1] #[4, 5, 6, 7]

In [19]:
event_frame = parse_bundle_events(experiment_path, experiment_ids)

Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/1/1
Computing time delta
Parsing done


/home/sterz/maci-docker-compose/maci_data/eval/data_handlers/runtimes.py:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  time_df = time_df.append(instance)


In [20]:
for bundle, group in event_frame[(event_frame["Event"] == "creation")].groupby("Bundle"):
    if len(group) != 1:
        print(bundle)
        print(group)

In [21]:
event_frame

,Simulation ID,Payload Size,Timestamp,Event,Node,Bundle,Software,Bundles per Second,CLA,# Nodes,# Payloads,Relative Timestamp
590,1,1000,1900-01-01 10:46:36.593,creation,n1,2C194C195137397DF2CAA13645839B737E9E627B140F04...,Serval,10,default,32,10,0 days 00:00:00
591,1,1000,1900-01-01 10:46:36.604,sending,n1,2C194C195137397DF2CAA13645839B737E9E627B140F04...,Serval,10,default,32,10,0 days 00:00:00.011000
592,1,1000,1900-01-01 10:46:36.695,creation,n1,D8A4447064E80E6B7D719EAB839165F628E0CD9EDA837D...,Serval,10,default,32,10,0 days 00:00:00.102000
593,1,1000,1900-01-01 10:46:36.705,sending,n1,D8A4447064E80E6B7D719EAB839165F628E0CD9EDA837D...,Serval,10,default,32,10,0 days 00:00:00.112000
594,1,1000,1900-01-01 10:46:36.796,creation,n1,9DBF4CBEB2F7105B4C5214DA1F13DD4E7FC0E6984BC084...,Serval,10,default,32,10,0 days 00:00:00.203000
...,...,...,...,...,...,...,...,...,...,...,...,...
505,1,1000,1900-01-01 10:46:43.610,delivery,n32,D8A4447064E80E6B7D719EAB839165F628E0CD9EDA837D...,Serval,10,default,32,10,0 days 00:00:07.017000
506,1,1000,1900-01-01 10:46:43.616,delivery,n32,0E5F3573A39E305BF9BE84D98C3C496707FE9CD25575D2...,Serval,10,default,32,10,0 days 00:00:07.023000
507,1,1000,1900-01-01 10:46:43.623,delivery,n32,50506D2E791FDDBE4D8F466AEB8E147EC51A52C2F81F2A...,Serval,10,default,32,10,0 days 00:00:07.030000
508,1,1000,1900-01-01 10:46:43.629,delivery,n32,73C14D3F743DF81C18904F8A1602054EE4C614BD3FAE1D...,Serval,10,default,32,10,0 days 00:00:07.036000
